In [1]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr

In [2]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test = test.filter(regex=r'feature', axis=1)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class MLPRegressorTorch(nn.Module):
    def __init__(self, input_size, hidden1, hidden2, hidden3, drop1, drop2, drop3):
        super(MLPRegressorTorch,self).__init__()
        self.hidden1 = nn.Linear(input_size, hidden1)
        self.drop1 = nn.Dropout(drop1)
        self.hidden2 = nn.Linear(hidden1, hidden2)
        self.drop2 = nn.Dropout(drop2)
        self.hidden3 = nn.Linear(hidden2, hidden3)
        self.drop3 = nn.Dropout(drop3)
        self.out = nn.Linear(hidden3, 1)
        
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.drop1(x)
        x = F.relu(self.hidden2(x))
        x = self.drop2(x)
        x = F.relu(self.hidden3(x))
        x = self.drop3(x)
        return self.out(x)
    
    
#MLPRegressorTorch(X_train0.shape[1], 10, 10)(torch.from_numpy(X_train0.values).float())

In [4]:
from skopt import gp_minimize



X_train0_t = torch.from_numpy(X_train0.values).float().cuda()
y_train0_t = torch.from_numpy(y_train0.values).float().unsqueeze(dim=-1).cuda()

X_train1_t = torch.from_numpy(X_train1.values).float().cuda()
X_val1_t = torch.from_numpy(X_val1.values).float().cuda()
X_test_t = torch.from_numpy(X_test.values).float().cuda()


def tune_mlp(params):
    #scaling?
    torch.manual_seed(0)
    hidden1, hidden2, hidden3, drop1, drop2, drop3, learning_rate = params
    mdl = MLPRegressorTorch(X_train0.shape[1], hidden1, hidden2, hidden3, drop1, drop2, drop3).cuda()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(mdl.parameters(), lr=learning_rate)
    
    for epoch in range(300): 
        optimizer.zero_grad()

        p = mdl(X_train0_t)
        loss = criterion(p, y_train0_t)
        loss.backward()
        optimizer.step()
    
    p = mdl(X_train1_t).detach().cpu().numpy()
    model_name_train1 = "./preds_train1/mlp_{}_{}_{}_{}_{}_{}_{}.pkl.z".format(hidden1, 
                                                                   hidden2, 
                                                                   hidden3,
                                                                   drop1,
                                                                   drop2,
                                                                   drop3,
                                                                   learning_rate) 
    jb.dump(p, model_name_train1)
    #print(p)
    
    metric = spearmanr(y_train1.values,p).correlation
    
    p = mdl(X_val1_t).detach().cpu().numpy()
    model_name_val1 = "./preds_val1/mlp_{}_{}_{}_{}_{}_{}_{}.pkl.z".format(hidden1, 
                                                                   hidden2, 
                                                                   hidden3,
                                                                   drop1,
                                                                   drop2,
                                                                   drop3,
                                                                   learning_rate)
    jb.dump(p, model_name_val1)
    
    p = mdl(X_test_t).detach().cpu().numpy()
    model_name_test = "./preds_test/mlp_{}_{}_{}_{}_{}_{}_{}.pkl.z".format(hidden1, 
                                                                   hidden2, 
                                                                   hidden3,
                                                                   drop1,
                                                                   drop2,
                                                                   drop3,
                                                                   learning_rate) 
    jb.dump(p, model_name_test)
    
    print(params, metric)
    print()
    
    return -metric

In [ ]:
#space = [(10, 500),
#         (10, 500),
#         (10, 500),
#         (0.0, 0.95),
#         (0.0, 0.95),
#         (0.0, 0.95),
#         (1e-5, 1e-3, 'log-uniform')]


space = [(500, 2000),
         (500, 2000),
         (500, 2000),
         (0.0, 0.95),
         (0.0, 0.95),
         (0.0, 0.95),
         (1e-6, 1e-3, 'log-uniform')]

res = gp_minimize(tune_mlp, space, random_state=0, verbose=1, n_calls=20)

In [ ]:
# fazer diário